# Toronto Neighborhoods: K-Means Clustering

### Loading Libraries 

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## Scraping and cleaning data

In [6]:
dfTor = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
dfTor.rename(columns={"Postcode": "PostalCode", "Neighbourhood":"Neighborhood"}, inplace = True) #change name of the postcode column 
dfTor = dfTor[dfTor.Borough != 'Not assigned'] #ignore boroughs with Not Assigned 
print("Shape of the data frame", dfTor.shape )
dfTor.head(20)

Shape of the data frame (210, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


replace not assigned neighborhoods with bourough name 

In [7]:

dfTor['Neighborhood'].mask(dfTor['Neighborhood'] == 'Not assigned', dfTor['Borough'], inplace=True) 
print("Shape of the data frame", dfTor.shape )

Shape of the data frame (210, 3)


#### Merge neighboorhoods in the same postal code

In [62]:
dfTor1 = dfTor.groupby(['PostalCode'], sort=False).agg({'Borough': 'first','Neighborhood': ', '.join})
dfTor1.reset_index(inplace = True) 
dfTor1.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


### Get shape of the notebook 

In [63]:
dfTor1.shape

(103, 3)

## Add geospatial data to the Dataframe

#### loading lat and long data 
geocode not working, will use csv file in the project folder

In [64]:
# The code was removed by Watson Studio for sharing.

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Assign lat and long data to the neighbrohood

In [66]:
longs= []
lats=[]
for index, row in dfTor1.iterrows():
    locRow = df_data_1.loc[df_data_1['PostalCode'] == row['PostalCode']]
    longs.append(float(locRow['Latitude']))
    lats.append(float(locRow['Longitude']))
    
    
dfTor1["Lattitude"]=lats
dfTor1["Longitude"]=longs
dfTor1.head()


,PostalCode,Borough,Neighborhood,Lattitude,Longitude
0,M3A,North York,Parkwoods,-79.329656,43.753259
1,M4A,North York,Victoria Village,-79.315572,43.725882
2,M5A,Downtown Toronto,Harbourfront,-79.360636,43.654260
3,M6A,North York,"Lawrence Heights, Lawrence Manor",-79.464763,43.718518
4,M7A,Downtown Toronto,Queen's Park,-79.389494,43.662301
